In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수

In [2]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 시드값

In [3]:
SEED = 42

- 데이터 불러오기

In [4]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [5]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2310), (12225, 2310))

# 결측치 처리

In [6]:
train_ft.isnull().sum()

,0
ID,0
총방문일수,0
첫구매날짜,0
마지막구매날짜,0
백화점이용기간,0
...,...
계절_구매변동성,0
구매주기_대비_구매금액,2
방문당_평균구매금액,2
구매금액_대비_환불금액,0


In [7]:
test_ft.isnull().sum()

,0
ID,0
총방문일수,0
첫구매날짜,0
마지막구매날짜,0
백화점이용기간,0
...,...
계절_구매변동성,0
구매주기_대비_구매금액,3
방문당_평균구매금액,3
구매금액_대비_환불금액,0


In [8]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
pivot_지점코드_A112000,2
pivot_지점코드_A144000,2
pivot_지점코드_A202000,2
pivot_지점코드_A373000,2
pivot_중분류_DC캐주얼,2
...,...
Tuesday_sum,2
Wednesday_sum,2
방문일수_대비_구매건수,2
구매주기_대비_구매금액,2


In [9]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
pivot_지점코드_A112000,3
pivot_지점코드_A144000,3
pivot_지점코드_A202000,3
pivot_지점코드_A373000,3
pivot_중분류_DC캐주얼,3
...,...
Tuesday_sum,3
Wednesday_sum,3
방문일수_대비_구매건수,3
구매주기_대비_구매금액,3


In [10]:
buy_cash = train_ft["구매주기_대비_구매금액"].mean()
buy_cash

399588.32009227417

In [11]:
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매주기_대비_구매금액"].fillna(buy_cash)
train_ft["방문당_평균구매금액"]= train_ft["방문당_평균구매금액"].fillna(0)
train_ft.fillna(method='ffill', inplace=True)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft.fillna(method='ffill', inplace=True)

<ipython-input-11-e22e55b1b7a1>:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ft.fillna(method='ffill', inplace=True)
<ipython-input-11-e22e55b1b7a1>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_ft.fillna(method='ffill', inplace=True)


In [12]:
 if train_ft.isnull().sum().sum() > 0:
    print(train_ft.columns)

# one hot 인코딩

In [13]:
#train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

In [14]:
#from sklearn.preprocessing import OneHotEncoder

In [15]:
#cols = train_ft.select_dtypes("object").columns #범주형 피처만 데이터프레임으로 출력
#cols

In [16]:
#enc = OneHotEncoder(handle_unknown="ignore")
#enc.fit(train_ft[cols])

In [17]:
#enc_cols = enc.get_feature_names_out()
#enc_cols

In [18]:
#train_ft[cols]

In [19]:
#train_ft[enc_cols] = enc.transform(train_ft[cols]).toarray()
#test_ft[enc_cols] = enc.transform(test_ft[cols]).toarray()

In [20]:
#train_ft.shape, test_ft.shape

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [21]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2309), (12225, 2309))

- 추가 피처 만들어 보기

In [22]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

In [23]:
train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2312), (12225, 2312))

## Feature Encoding

In [24]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
첫구매날짜,338
마지막구매날짜,335
주구매지점,5
주구매_중분류,231
주구매_대분류,29
고객등급,5


In [25]:
%pip install category_encoders

In [26]:
import category_encoders as ce

In [27]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","주구매_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","주구매_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 2346), (12225, 2346))

In [28]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 2347), (12225, 2347))

- 문자열 피처 삭제

In [29]:
cols

['첫구매날짜', '마지막구매날짜', '주구매지점', '주구매_중분류', '주구매_대분류', '고객등급']

In [30]:
train_ft = train_ft.drop(columns=cols, errors='ignore')
test_ft = test_ft.drop(columns=cols,errors='ignore')
train_ft.shape, test_ft.shape

((14940, 2341), (12225, 2341))

In [31]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [33]:
# train_ft와 test_ft의 공통 열만 선택
#common_columns = train_ft.columns.intersection(test_ft.columns)

# 공통 열만을 남긴 train_ft와 test_ft
#train_ft_common = train_ft[common_columns]
#test_ft_common = test_ft[common_columns]

# 스케일러 초기화 및 변환
#scaler = StandardScaler()
#train_ft[common_columns] = scaler.fit_transform(train_ft_common)
#test_ft[common_columns] = scaler.transform(test_ft_common)

# 결과 확인
#print(train_ft.head())
#print(test_ft.head())

In [34]:
train_ft.head()

,총방문일수,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,평균구매요일,주구매요일,월요일_구매비율,...,주구매_대분류_21,주구매_대분류_22,주구매_대분류_23,주구매_대분류_24,주구매_대분류_25,주구매_대분류_26,주구매_대분류_27,주구매_대분류_28,주구매_대분류_29,주구매_중분류_cnt
0,9.0,255.0,28.0,2.0,1.444444,22.934084,104.0,3.000000,2.0,0.076923,...,0,0,0,0,0,0,0,0,0,233
1,21.0,352.0,16.0,1.0,2.000000,11.739134,3.0,2.309524,3.0,0.047619,...,0,0,0,0,0,0,0,0,0,511
2,54.0,356.0,6.0,11.0,1.777778,5.363052,4.0,2.322917,0.0,0.260417,...,0,0,0,0,0,0,0,0,0,13
3,86.0,345.0,4.0,17.0,2.034884,3.563793,10.0,3.074286,4.0,0.068571,...,0,0,0,0,0,0,0,0,0,4693
4,24.0,313.0,13.0,6.0,1.458333,12.037858,44.0,3.800000,4.0,0.085714,...,0,0,0,0,0,0,0,0,0,2080


In [35]:
# 무한대 값을 NaN으로 대체
train_ft.replace([np.inf, -np.inf], np.nan, inplace=True)
test_ft.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값 확인 후 처리 (필요에 따라 제거하거나 대체)
train_ft.fillna(train_ft.mean(), inplace=True)  # 평균으로 NaN 대체
test_ft.fillna(test_ft.mean(), inplace=True)    # 평균으로 NaN 대체

# 스케일러 초기화 및 변환
scaler = StandardScaler()
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [36]:
#train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
#test_ft[test_ft.columns] = scaler.transform(test_ft)
#train_ft.head()

# 정답 데이터

In [37]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# cv 점수 확인해보기

In [38]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

In [39]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
np.mean(scores)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


0.7236049102472087

# 모델 학습

In [40]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.205851 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43785
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1643
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# 테스트 데이터 예측

In [41]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [42]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [43]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,0.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


- 예측 결과를 csv 파일로 저장하여 제출

In [44]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)